<a href="https://colab.research.google.com/github/Shritej24c/Credit-Risk/blob/main/XGB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install dask

### Importing Packages

In [ ]:
import pandas as pd 
import numpy as np
import xgboost as xgb
import time as t
from sklearn.metrics import roc_auc_score

In [ ]:
pd.set_option('display.max_columns', None)

### Reading CSVs

In [ ]:
train = pd.read_csv("train_data.csv")
train.head()

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_42,D_43,D_44,B_4,D_45,B_5,R_2,D_46,D_47,D_48,D_49,B_6,B_7,B_8,D_50,D_51,B_9,R_3,D_52,P_3,B_10,D_53,S_5,B_11,S_6,D_54,R_4,S_7,B_12,S_8,D_55,D_56,B_13,R_5,D_58,S_9,B_14,D_59,D_60,D_61,B_15,S_11,D_62,D_63,D_64,D_65,B_16,B_17,B_18,B_19,D_66,B_20,D_68,S_12,R_6,S_13,B_21,D_69,B_22,D_70,D_71,D_72,S_15,B_23,D_73,P_4,D_74,D_75,D_76,B_24,R_7,D_77,B_25,B_26,D_78,D_79,R_8,R_9,S_16,D_80,R_10,R_11,B_27,D_81,D_82,S_17,R_12,B_28,R_13,D_83,R_14,R_15,D_84,R_16,B_29,B_30,S_18,D_86,D_87,R_17,R_18,D_88,B_31,S_19,R_19,B_32,S_20,R_20,R_21,B_33,D_89,R_22,R_23,D_91,D_92,D_93,D_94,R_24,R_25,D_96,S_22,S_23,S_24,S_25,S_26,D_102,D_103,D_104,D_105,D_106,D_107,B_36,B_37,R_26,R_27,B_38,D_108,D_109,D_110,D_111,B_39,D_112,B_40,S_27,D_113,D_114,D_115,D_116,D_117,D_118,D_119,D_120,D_121,D_122,D_123,D_124,D_125,D_126,D_127,D_128,D_129,B_41,B_42,D_130,D_131,D_132,D_133,R_28,D_134,D_135,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-03-09,0.938469,0.001733,0.008724,1.006838,0.009228,0.124035,0.008771,0.004709,NaN,NaN,0.000630,0.080986,0.708906,0.170600,0.006204,0.358587,0.525351,0.255736,NaN,0.063902,0.059416,0.006466,0.148698,1.335856,0.008207,0.001423,0.207334,0.736463,0.096219,NaN,0.023381,0.002768,0.008322,1.001519,0.008298,0.161345,0.148266,0.922998,0.354596,0.152025,0.118075,0.001882,0.158612,0.065728,0.018385,0.063646,0.199617,0.308233,0.016361,0.401619,0.091071,CR,O,0.007126,0.007665,NaN,0.652984,0.008520,NaN,0.004730,6.0,0.272008,0.008363,0.515222,0.002644,0.009013,0.004808,0.008342,0.119403,0.004802,0.108271,0.050882,NaN,0.007554,0.080422,0.069067,NaN,0.004327,0.007562,NaN,0.007729,0.000272,0.001576,0.004239,0.001434,NaN,0.002271,0.004061,0.007121,0.002456,0.002310,0.003532,0.506612,0.008033,1.009825,0.084683,0.003820,0.007043,0.000438,0.006452,0.000830,0.005055,NaN,0.0,0.005720,0.007084,NaN,0.000198,0.008907,NaN,1,0.002537,0.005177,0.006626,0.009705,0.007782,0.002450,1.001101,0.002665,0.007479,0.006893,1.503673,1.006133,0.003569,0.008871,0.003950,0.003647,0.004950,0.894090,0.135561,0.911191,0.974539,0.001243,0.766688,1.008691,1.004587,0.893734,NaN,0.670041,0.009968,0.004572,NaN,1.008949,2.0,NaN,0.004326,NaN,NaN,NaN,1.007336,0.210060,0.676922,0.007871,1.0,0.238250,0.0,4.0,0.232120,0.236266,0.0,0.702280,0.434345,0.003057,0.686516,0.008740,1.0,1.003319,1.007819,1.000080,0.006805,NaN,0.002052,0.005972,NaN,0.004345,0.001535,NaN,NaN,NaN,NaN,NaN,0.002427,0.003706,0.003818,NaN,0.000569,0.000610,0.002674
1,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-04-07,0.936665,0.005775,0.004923,1.000653,0.006151,0.126750,0.000798,0.002714,NaN,NaN,0.002526,0.069419,0.712795,0.113239,0.006206,0.353630,0.521311,0.223329,NaN,0.065261,0.057744,0.001614,0.149723,1.339794,0.008373,0.001984,0.202778,0.720886,0.099804,NaN,0.030599,0.002749,0.002482,1.009033,0.005136,0.140951,0.143530,0.919414,0.326757,0.156201,0.118737,0.001610,0.148459,0.093935,0.013035,0.065501,0.151387,0.265026,0.017688,0.406326,0.086805,CR,O,0.002413,0.007148,NaN,0.647093,0.002238,NaN,0.003879,6.0,0.188970,0.004030,0.509048,0.004193,0.007842,0.001283,0.006524,0.140611,0.000094,0.101018,0.040469,NaN,0.004832,0.081413,0.074166,NaN,0.004203,0.005304,NaN,0.001864,0.000979,0.009896,0.007597,0.000509,NaN,0.009810,0.000127,0.005966,0.000395,0.001327,0.007773,0.500855,0.000760,1.009461,0.081843,0.000347,0.007789,0.004311,0.002332,0.009469,0.003753,NaN,0.0,0.007584,0.006677,NaN,0.001142,0.005907,NaN,1,0.008427,0.008979,0.001854,0.009924,0.005987,0.002247,1.006779,0.002508,0.006827,0.002837,1.503577,1.005791,0.000571,0.000391,0.008351,0.008850,0.003180,0.902135,0.136333,0.919876,0.975624,0.004561,0.786007,1.000084,1.004118,0.906841,NaN,0.668647,0.003921,0.004654,NaN,1.003205,2.0,NaN,0.008707,NaN,NaN,NaN,1.007653,0.184093,0.822281,0.003444,1.0,0.247217,0.0,4.0,0.243532,0.241885,0.0,0.707017,0.430501,0.001306,0.686414,0.000755,1.0,1.008394,1.004333,1.008344,0.004407,NaN,0.001034,0.004838,NaN,0.

In [ ]:
train.shape

(5531451, 190)

In [ ]:
labels = pd.read_csv("train_labels.csv")
labels.head()

,customer_ID,target
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,0
2,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,0
3,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...,0
4,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,0


In [ ]:
train['S_2'] = pd.to_datetime(train['S_2'])

In [ ]:
df = pd.merge(train, labels, on = 'customer_ID', how = 'left')
df.head()

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_42,D_43,D_44,B_4,D_45,B_5,R_2,D_46,D_47,D_48,D_49,B_6,B_7,B_8,D_50,D_51,B_9,R_3,D_52,P_3,B_10,D_53,S_5,B_11,S_6,D_54,R_4,S_7,B_12,S_8,D_55,D_56,B_13,R_5,D_58,S_9,B_14,D_59,D_60,D_61,B_15,S_11,D_62,D_63,D_64,D_65,B_16,B_17,B_18,B_19,D_66,B_20,D_68,S_12,R_6,S_13,B_21,D_69,B_22,D_70,D_71,D_72,S_15,B_23,D_73,P_4,D_74,D_75,D_76,B_24,R_7,D_77,B_25,B_26,D_78,D_79,R_8,R_9,S_16,D_80,R_10,R_11,B_27,D_81,D_82,S_17,R_12,B_28,R_13,D_83,R_14,R_15,D_84,R_16,B_29,B_30,S_18,D_86,D_87,R_17,R_18,D_88,B_31,S_19,R_19,B_32,S_20,R_20,R_21,B_33,D_89,R_22,R_23,D_91,D_92,D_93,D_94,R_24,R_25,D_96,S_22,S_23,S_24,S_25,S_26,D_102,D_103,D_104,D_105,D_106,D_107,B_36,B_37,R_26,R_27,B_38,D_108,D_109,D_110,D_111,B_39,D_112,B_40,S_27,D_113,D_114,D_115,D_116,D_117,D_118,D_119,D_120,D_121,D_122,D_123,D_124,D_125,D_126,D_127,D_128,D_129,B_41,B_42,D_130,D_131,D_132,D_133,R_28,D_134,D_135,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145,target
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-03-09,0.938469,0.001733,0.008724,1.006838,0.009228,0.124035,0.008771,0.004709,NaN,NaN,0.000630,0.080986,0.708906,0.170600,0.006204,0.358587,0.525351,0.255736,NaN,0.063902,0.059416,0.006466,0.148698,1.335856,0.008207,0.001423,0.207334,0.736463,0.096219,NaN,0.023381,0.002768,0.008322,1.001519,0.008298,0.161345,0.148266,0.922998,0.354596,0.152025,0.118075,0.001882,0.158612,0.065728,0.018385,0.063646,0.199617,0.308233,0.016361,0.401619,0.091071,CR,O,0.007126,0.007665,NaN,0.652984,0.008520,NaN,0.004730,6.0,0.272008,0.008363,0.515222,0.002644,0.009013,0.004808,0.008342,0.119403,0.004802,0.108271,0.050882,NaN,0.007554,0.080422,0.069067,NaN,0.004327,0.007562,NaN,0.007729,0.000272,0.001576,0.004239,0.001434,NaN,0.002271,0.004061,0.007121,0.002456,0.002310,0.003532,0.506612,0.008033,1.009825,0.084683,0.003820,0.007043,0.000438,0.006452,0.000830,0.005055,NaN,0.0,0.005720,0.007084,NaN,0.000198,0.008907,NaN,1,0.002537,0.005177,0.006626,0.009705,0.007782,0.002450,1.001101,0.002665,0.007479,0.006893,1.503673,1.006133,0.003569,0.008871,0.003950,0.003647,0.004950,0.894090,0.135561,0.911191,0.974539,0.001243,0.766688,1.008691,1.004587,0.893734,NaN,0.670041,0.009968,0.004572,NaN,1.008949,2.0,NaN,0.004326,NaN,NaN,NaN,1.007336,0.210060,0.676922,0.007871,1.0,0.238250,0.0,4.0,0.232120,0.236266,0.0,0.702280,0.434345,0.003057,0.686516,0.008740,1.0,1.003319,1.007819,1.000080,0.006805,NaN,0.002052,0.005972,NaN,0.004345,0.001535,NaN,NaN,NaN,NaN,NaN,0.002427,0.003706,0.003818,NaN,0.000569,0.000610,0.002674,0
1,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-04-07,0.936665,0.005775,0.004923,1.000653,0.006151,0.126750,0.000798,0.002714,NaN,NaN,0.002526,0.069419,0.712795,0.113239,0.006206,0.353630,0.521311,0.223329,NaN,0.065261,0.057744,0.001614,0.149723,1.339794,0.008373,0.001984,0.202778,0.720886,0.099804,NaN,0.030599,0.002749,0.002482,1.009033,0.005136,0.140951,0.143530,0.919414,0.326757,0.156201,0.118737,0.001610,0.148459,0.093935,0.013035,0.065501,0.151387,0.265026,0.017688,0.406326,0.086805,CR,O,0.002413,0.007148,NaN,0.647093,0.002238,NaN,0.003879,6.0,0.188970,0.004030,0.509048,0.004193,0.007842,0.001283,0.006524,0.140611,0.000094,0.101018,0.040469,NaN,0.004832,0.081413,0.074166,NaN,0.004203,0.005304,NaN,0.001864,0.000979,0.009896,0.007597,0.000509,NaN,0.009810,0.000127,0.005966,0.000395,0.001327,0.007773,0.500855,0.000760,1.009461,0.081843,0.000347,0.007789,0.004311,0.002332,0.009469,0.003753,NaN,0.0,0.007584,0.006677,NaN,0.001142,0.005907,NaN,1,0.008427,0.008979,0.001854,0.009924,0.005987,0.002247,1.006779,0.002508,0.006827,0.002837,1.503577,1.005791,0.000571,0.000391,0.008351,0.008850,0.003180,0.902135,0.136333,0.919876,0.975624,0.004561,0.786007,1.000084,1.004118,0.906841,NaN,0.668647,0.003921,0.004654,NaN,1.003205,2.0,NaN,0.008707,NaN,NaN,NaN,1.007653,0.184093,0.822281,0.003444,1.0,0.247217,0.0,4.0,0.243532,0.241885,0.0,0.707017,0.430501,0.001306,0.686414,0.000755,1.0,1.008394,1.004333,1.008344,0.004407,NaN,0.001034,0.0048

In [ ]:
df.shape

(5531451, 191)

In [ ]:
len(df['customer_ID'].unique())

458913

In [ ]:
df.loc[df['customer_ID'] == df.iloc[1,0], :].shape

(13, 191)

In [ ]:
5531451/458913

12.053376130116165

### Randomly Selecting 1 month of data

In [ ]:
df12 = df.sample(frac=1, random_state = 21)  #shuffling the data
df1 = df12.groupby('customer_ID',as_index=False).first()
print("Min :", df1['S_2'].min(), "Max: ", df1['S_2'].max())
df1.shape

(415602, 190)

In [ ]:
df1.set_index('S_2').groupby(pd.Grouper(freq = 'M')).count()

,customer_ID,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_42,D_43,D_44,B_4,D_45,B_5,R_2,D_46,D_47,D_48,D_49,B_6,B_7,B_8,D_50,D_51,B_9,R_3,D_52,P_3,B_10,D_53,S_5,B_11,S_6,D_54,R_4,S_7,B_12,S_8,D_55,D_56,B_13,R_5,D_58,S_9,B_14,D_59,D_60,D_61,B_15,S_11,D_62,D_63,D_64,D_65,B_16,B_17,B_18,B_19,D_66,B_20,D_68,S_12,R_6,S_13,B_21,D_69,B_22,D_70,D_71,D_72,S_15,B_23,D_73,P_4,D_74,D_75,D_76,B_24,R_7,D_77,B_25,B_26,D_78,D_79,R_8,R_9,S_16,D_80,R_10,R_11,B_27,D_81,D_82,S_17,R_12,B_28,R_13,D_83,R_14,R_15,D_84,R_16,B_29,B_30,S_18,D_86,D_87,R_17,R_18,D_88,B_31,S_19,R_19,B_32,S_20,R_20,R_21,B_33,D_89,R_22,R_23,D_91,D_92,D_93,D_94,R_24,R_25,D_96,S_22,S_23,S_24,S_25,S_26,D_102,D_103,D_104,D_105,D_106,D_107,B_36,B_37,R_26,R_27,B_38,D_108,D_109,D_110,D_111,B_39,D_112,B_40,S_27,D_113,D_114,D_115,D_116,D_117,D_118,D_119,D_120,D_121,D_122,D_123,D_124,D_125,D_126,D_127,D_128,D_129,B_41,B_42,D_130,D_131,D_132,D_133,R_28,D_134,D_135,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145,target
S_2,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017-03-31,30549,30424,30549,30549,30549,30549,26539,30549,30549,4471,24756,29400,30549,30549,30549,30549,26370,30549,28671,3820,30549,30549,30538,14590,30549,30549,30549,30495,30423,30549,9860,30549,30549,30549,30549,30549,26539,30549,30549,30488,17253,30549,30549,30549,20626,30549,30498,30549,29025,30509,30549,28606,30549,30545,30549,30549,16933,30549,30549,3937,30549,30545,30549,30549,30549,30549,30545,30549,30468,30549,30495,30549,30549,295,30549,30471,30549,4049,30549,30549,24030,30509,30549,29400,30451,30549,2430,30549,30471,30549,30549,30549,30517,8879,30549,30549,30549,30549,30545,30549,30549,30495,30549,2751,30549,30549,30549,63,30549,30549,178,30549,30549,30549,30549,30549,30549,30549,30549,30495,30549,30549,30512,30549,30549,30549,30549,30549,30549,30546,30549,30546,30547,30549,30549,30449,30449,15670,3813,30449,30549,30549,3152,30126,30549,1415,30549,308,308,311,30549,30549,26028,30549,30549,30549,30549,30549,30549,30549,30549,30549,30549,30549,30549,30549,30549,30549,30449,30449,30549,443,30449,30449,3820,30549,30549,2154,2154,2154,2154,2154,30449,30549,30449,5410,30449,30549,30449,30549
2017-04-30,31263,31135,31263,31263,31263,31263,27249,31263,31263,5077,25265,30111,31263,31263,31263,31263,27028,31263,29337,4018,31263,31263,31237,14950,31263,31263,31263,31197,31132,31263,10191,31263,31263,31263,31263,31263,27249,31263,31263,31189,17645,31263,31263,31263,21209,31263,31204,31263,29702,31207,31263,29257,31263,31258,31263,31263,17233,31263,31263,4162,31263,31258,31263,31263,31263,31263,31259,31263,31167,31263,31197,31263,31263,366,31263,31169,31263,4130,31263,31263,24773,31207,31263,30111,31159,31263,2468,31263,31169,31263,31263,31263,31218,8942,31263,31263,31263,31263,31259,31263,31263,31197,31263,2742,31263,31263,31263,66,31263,31263,157,31263,31263,31263,31263,31263,31263,31263,31263,31197,31263,31263,31214,31263,31263,31263,31263,31263,31263,31263,31263,31263,31262,31263,31263,31158,31158,16186,4016,31158,31263,31263,3315,30880,31263,1416,31263,319,319,321,31263,31263,26691,31263,31263,31263,31263,31263,31263,31263,31263,31263,31263,31263,31263,31263,31263,31263,31158,31158,31263,506,31158,31158,4018,31263,31263,2279,2279,2279,2279,2279,31158,31263,31158,5537,31158,31263,31158,31263
2017-05-31,30904,30793,30904,30904,30904,30904,26962,30904,30904,5191,24994,29739,30904,30904,30904,30904,26683,30904,28989,4045,30904,30904,30891,14710,30904,30904,30904,30859,30790,30904,10217,30904,30904,30904,30904,30904,26962,30904,30904,30856,17452,30904,30904,30904,21057,30904,30852,30904,29420,30857,30904,28936,30904,30904,30904,30904,17234,30904,30904,4231,30904,30904,30904,30904,30904,30904,30904,30904,30836,30904,30859,30904,30904,370,30904,30836,30904,4052,30904,30904,24374,30857,30904,29739,30819,30904,2578,30904,30836,30904,30904,30904,30878,8861,30904,30904,30904,30904,3

In [ ]:
df1.isnull().sum()

customer_ID         0
S_2                 0
P_2              2434
D_39                0
B_1                 0
                ...  
D_142          373333
D_143            2532
D_144               0
D_145            2532
target              0
Length: 191, dtype: int64

In [ ]:
for i in df1.columns:
    if df1[i].dtypes != 'float64':
        print(i, df_rnd1[i].dtypes)

customer_ID object
S_2 datetime64[ns]
D_63 object
D_64 object
B_31 int64


#### Converting Categorical Variables to dummy variables

In [ ]:
print("Min: ", df1['S_2'].min(), "Max: ", df1['S_2'].max())

Min:  2017-03-01 00:00:00 Max:  2018-03-31 00:00:00


In [ ]:
df_new = pd.get_dummies(df1, columns = ['D_63', 'D_64'], drop_first = True)
df_new.to_csv('final_df.csv')

### Split Dataset in Train, Test1 & Test2

In [ ]:
train_df = df_new.loc[(df_new['S_2'] >= '2017-05') & (df_new['S_2'] <= '2018-01'), :]
test1_df = df_new.loc[(df_new['S_2'] <= '2017-04') , :]
test2_df = df_new.loc[ (df_new['S_2'] >= '2018-02'), :]

In [ ]:
train_df.shape

(269980, 197)

In [ ]:
xtrain = train_df.drop(columns = ['target', 'customer_ID', 'S_2'])
ytrain = train_df['target']

In [ ]:
xtrain.shape

(269980, 194)

In [ ]:
t1 = t.time()
m1 = xgb.XGBClassifier(random_state = 21)
m1.fit(xtrain, ytrain)
feat_imp = pd.DataFrame({'columns': xtrain.columns, 'feat_imp': m1.feature_importances_})
feat_imp.loc[feat_imp['feat_imp'] > 0.005,:].sort_values(['feat_imp'], ascending = False)
feat_imp.to_csv('feat_imp.csv')
t2 = t.time()
print(t2-t1)

210.37714004516602


In [ ]:
help(xgb.XGBClassifier())

Help on XGBClassifier in module xgboost.sklearn object:

class XGBClassifier(XGBModel, sklearn.base.ClassifierMixin)
 |  XGBClassifier(*, objective: Union[str, Callable[[numpy.ndarray, numpy.ndarray], Tuple[numpy.ndarray, numpy.ndarray]], NoneType] = 'binary:logistic', use_label_encoder: Union[bool, NoneType] = None, **kwargs: Any) -> None
 |  
 |  Implementation of the scikit-learn API for XGBoost classification.
 |  
 |  
 |  Parameters
 |  ----------
 |  
 |      n_estimators : int
 |          Number of boosting rounds.
 |  
 |      max_depth :  Optional[int]
 |          Maximum tree depth for base learners.
 |      max_leaves :
 |          Maximum number of leaves; 0 indicates no limit.
 |      max_bin :
 |          If using histogram-based algorithm, maximum number of bins per feature
 |      grow_policy :
 |          Tree growing policy. 0: favor splitting at nodes closest to the node, i.e. grow
 |          depth-wise. 1: favor splitting at nodes with highest loss change.
 |     

In [ ]:
t1 = t.time()
m2 = xgb.XGBClassifier(n_estimators = 300, learning_rate = 0.5, max_depth = 4, subsample = .5, colsample_bytree = 0.5, scale_pos_weight = 5, random_state =21 )
m2.fit(xtrain, ytrain)
feat_imp1 = pd.DataFrame({'columns': xtrain.columns, 'feat_imp': m2.feature_importances_})
feat_imp1.loc[feat_imp1['feat_imp'] > 0.005,:].sort_values(['feat_imp'], ascending = False)
feat_imp1.to_csv('feat_imp1.csv')
t2 = t.time()
print(t2-t1)

159.01183223724365


In [ ]:
start = t.time()
df_rnd1 = train.loc[(train['S_2'].dt.year == 2017) & (train['S_2'].dt.month == 7), :]
print(df_rnd1.shape)
end = t.time()
print(end - start)

(415602, 190)
31.137080669403076


In [ ]:
feat_imp1.loc[feat_imp1['feat_imp'] > 0.005, 'columns'].reset_index()

,index,columns
0,0,P_2
1,2,B_1
2,4,R_1
3,5,S_3
4,6,D_41
5,8,D_42
6,9,D_43
7,10,D_44
8,11,B_4
9,12,D_45


In [ ]:
feat_imp.loc[feat_imp1['feat_imp'] > 0.005, 'columns'].reset_index()

,index,columns
0,0,P_2
1,2,B_1
2,4,R_1
3,5,S_3
4,6,D_41
5,8,D_42
6,9,D_43
7,10,D_44
8,11,B_4
9,12,D_45


In [ ]:
col = list(set(feat_imp1.loc[feat_imp1['feat_imp'] > 0.005, 'columns'].to_list()).union(set(feat_imp.loc[feat_imp['feat_imp'] > 0.005, 'columns'].to_list())))
print(col)

['D_50', 'S_9', 'D_56', 'D_112', 'D_62', 'R_3', 'D_49', 'D_51', 'D_132', 'D_75', 'B_17', 'B_10', 'B_8', 'D_41', 'B_4', 'D_77', 'D_48', 'S_3', 'D_44', 'D_53', 'D_111', 'D_43', 'B_2', 'B_22', 'B_1', 'D_46', 'D_88', 'B_38', 'P_2', 'P_3', 'S_7', 'B_7', 'B_3', 'D_110', 'B_18', 'R_1', 'B_6', 'D_64_O', 'D_45', 'B_9', 'D_42', 'D_66', 'B_19', 'R_27', 'B_5']


In [ ]:
len(col)

45

In [ ]:
df = df_new.loc[:, col]
df.shape

(458913, 45)

In [ ]:
df_tr = df.loc[(df_new['S_2'] >= '2017-05') & (df_new['S_2'] <= '2018-01'), :]
df_ts1 = df.loc[(df_new['S_2'] <= '2017-04') , :]
df_ts2 = df.loc[ (df_new['S_2'] >= '2018-02'), :]

In [ ]:
grid_search = pd.DataFrame(columns = ['No. of Trees', 'LR', 'Subsample', '%features', 'Default Weight', 'AUC Train', 'AUC Test1', 'AUC Test2'])
num_trees = [50, 100, 500]
lr = [0.01, .1]
subsample = [.5, .8]
feat = [.5, 1]
def_w = [1, 5, 10]

for i in num_trees:
    for j in lr:
        for k in subsample:
            for f in feat:
                for d in def_w:
                    xgb_inst = xgb.XGBClassifier(n_estimators = i, learning_rate = j, subsample = k, colsample_bytree = f, )